## Create vegetation and land use type maps for each ecoregion

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import util
from tqdm import tqdm

In [3]:
#path to clean data folder
clean_data = "../data/clean/"
raw_data = "../data/raw/"

# Administrative boundaries
soco_bound = gpd.read_file(clean_data + "sonoma_county_boundary/sonoma_county_boundary.shp")
ca_state = gpd.read_file(clean_data + "ca_state_boundary/ca_state_boundary.shp")

# Sonoma Veg
sonoma_veg_simp = gpd.read_file(clean_data + "sonoma_veg/sonoma_veg.shp")
sonoma_veg_percs = pd.read_csv(clean_data + "simple_veg_all.csv")

# CA EPA eco regions
ecoregions = gpd.read_file(clean_data + "soco_ecoregion_l4/soco_ecoregion_l4.shp")

# Land use 
landuse = gpd.read_file(raw_data + "soco_landuse/soco_landuse.shp") 
landuse_percs = pd.read_csv(clean_data + "landuse_all.csv")
protected_ag = gpd.read_file(raw_data + "Vital_Lands_Data_Package/Base_Protected_Lands_Ag_Open_Space.shp") 
protected_cced = gpd.read_file(raw_data + "Vital_Lands_Data_Package/Base_Protected_Lands_CCED_2021b.shp").to_crs(epsg=2226)
protected_cpad = gpd.read_file(raw_data + "Vital_Lands_Data_Package/Base_Protected_Lands_CPAD_2021b_Holdings.shp").to_crs(epsg=2226)
urban_growth_boundaries = gpd.read_file(raw_data + "Vital_Lands_Data_Package/Base_Urban_Growth_Boundary.shp") 

# Critical Facilities
crit_facili = pd.read_csv(clean_data + "critical_facilities_stats.csv")

In [4]:
# clean the data
ecoregions_simp = util.simplify_ecoregions(ecoregions)
soco_bound = soco_bound.to_crs(epsg=2226)
ca_state = ca_state.to_crs(epsg=2226)
landuse_simp = util.longname_landuse(landuse)

In [21]:
ecoregions_simp

,l4_simple,geometry,ecoregion_acres
0,Bay Flats,"POLYGON ((6457375.369 1839801.158, 6458101.666...",31771.799491
1,Bodega Coastal Hills,"POLYGON ((6381082.225 1827591.674, 6379446.296...",89797.513014
2,Coastal Franciscan Redwood Forest,"POLYGON ((6199279.568 2057083.727, 6199127.431...",243527.483252
3,Fort Bragg/Fort Ross Terraces,"POLYGON ((6221139.364 1943186.149, 6221063.459...",24037.249709
4,Mayacmas Mountains,"POLYGON ((6343961.902 2057904.155, 6344919.458...",128342.895678
5,Napa-Sonoma-Lake Volcanic Highlands,"MULTIPOLYGON (((6400682.635 1951591.530, 64002...",124652.598843
6,Napa-Sonoma-Russian River Valleys,"POLYGON ((6271934.334 2073471.832, 6272744.838...",223194.335469
7,North Coast Range Eastern Slopes,"POLYGON ((6405821.549 1950568.891, 6405646.612...",13105.525900
8,Sonoma-Mendocino Mixed Forest,"POLYGON ((6266516.907 2073767.225, 6267442.639...",138833.291177


## MAPS

In [ ]:
# map of all ecoregions
# Get the extent of the eoi for mapping
xlim = ([ecoregions_simp.total_bounds[0] - 0.009e6,  ecoregions_simp.total_bounds[2] +  0.009e6])
ylim = ([ecoregions_simp.total_bounds[1] - 0.009e6,  ecoregions_simp.total_bounds[3] +  0.009e6])

# plot
fig, ax = plt.subplots(figsize=(15,15))
ca_state.plot(ax=ax, edgecolor = "grey", color = "#e2e2e2")
soco_bound.plot(ax=ax, edgecolor = "black", color = "None")
ecoregions_simp.plot(ax=ax, column = "l4_simple")
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('#add8e6')
ax

In [8]:
# Function to map the ecoregions
from typing import Union, Dict
def map_ecoregion_char(ecoregions: gpd.GeoDataFrame, characteristic: gpd.GeoDataFrame, which_eoi: str, cmap: Union[ListedColormap, Dict[str, str]], column: str):

    """Map each ecoregion characteristic

    Args:
        ecoregions (gpd.GeoDataFrame): shapefile of the ecoregions
        characteristic (gpd.GeoDataFrame): shapefile of vegetation types or landuse or other ecoregion characteristic
        which_eoi (str): The name of the ecoregion to run the function on
        cmap (ListedColormap): The color map to use 

    Returns:
        fig, ax: The figure 
    """

    # Get the eco region of of interest
    eoi = ecoregions[ecoregions["l4_simple"]==which_eoi]

    # intersect the veg map and the ecoregion of interest
    eoi_intersect = gpd.overlay(characteristic, eoi, how='intersection')

    # Get the extent of the eoi for mapping
    xlim = ([eoi.total_bounds[0] - 0.009e6,  eoi.total_bounds[2] +  0.009e6])
    ylim = ([eoi.total_bounds[1] - 0.009e6,  eoi.total_bounds[3] +  0.009e6])

    # plot the simplified sonoma veg data
    fig, ax = plt.subplots(figsize=(15,15))
    ca_state.plot(ax=ax, edgecolor = "grey", color = "#e2e2e2")

    if isinstance(cmap, dict):
        cmap = ListedColormap([cmap[x] for x in sorted(eoi_intersect[column].unique())])

    eoi_intersect.plot(ax=ax, column = column, cmap=cmap, legend = True)
    soco_bound.plot(ax=ax, edgecolor = "black", color = "None")
    ecoregions_simp.plot(ax=ax, edgecolor = "black", color = "None")
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor('#add8e6')
    return fig, ax 

In [9]:
# create list of ecoregions for looping through
ecoregion_list = ecoregions_simp.l4_simple.unique()

#### Sonoma Veg

In [12]:
# set the simplified veg map colors
colors = ["peru", "blue", "red", "grey", "green", "bisque", "yellow"]
veg_cmap=ListedColormap(colors)

In [ ]:
# loop through the ecoregions to create the veg map maps
for eco in tqdm(ecoregion_list):
    fig, _ = map_ecoregion_char(ecoregions_simp, sonoma_veg_simp, eco, veg_cmap, "natural_si")
    snake_case_eco = eco.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
    fig.savefig(f"../output/maps/{snake_case_eco}_veg.png", dpi=350)

#### Landuse

In [5]:
# landuse colors
landuse_colors = {
"Diverse Agriculture": "#7b5836",
"General Commercial": "#ff6600",
"General Industrial": "#fbc312",
"Limited Commercial": "#ffaf7a",
"Land Extensive Agriculture": "#D2B48C",
"Limited Industrial": "#fcfc64",
"Land Intensive Agriculture": "#422a14",
"Public / Quasi-public": "#006a4e",
"Rural Residential": "#80a3d8",
"Resources and Rural Development": "#6Eb440",
"Recreation and Visitor Serving Commercial": "#E79796",
"Urban Residential": "#1167b1"
}

In [ ]:
from matplotlib.colors import ListedColormap
land_cmap = ListedColormap([landuse_colors[x] for x in sorted(landuse_simp.landuse.unique())])
landuse_simp.plot(column = "landuse", figsize=(15,15), cmap=land_cmap, legend = True)

In [ ]:
from tqdm import tqdm
# loop through the ecoregions to create the landuse maps
for eco in tqdm(ecoregion_list):
    fig, _ = map_ecoregion_char(ecoregions_simp, landuse_simp, eco, landuse_colors, "landuse")
    snake_case_eco = eco.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
    fig.savefig(f"../output/maps/{snake_case_eco}_landuse.png", dpi=350)

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
soco_bound.plot(ax=ax, edgecolor = "black", color = "None")
#protected_ag.plot(ax=ax, column = "Holding_Ow", legend = True)
#protected_cced.plot(ax=ax, column = "eholdtyp", legend = True)
#protected_cpad.plot(ax=ax, column = "AGNCY_LEV", legend = True)
urban_growth_boundaries.plot(ax=ax, column = "NAME", legend = True)
ecoregions_simp.plot(ax=ax, edgecolor = "black", color = "None")
ax.set_xticks([])
ax.set_yticks([])
fig.savefig(f"../output/maps/urban_growth_boundaries_ecoregions.png", dpi=350)

## PIE CHARTS

In [16]:
# Create pie plot
from matplotlib.colors import ListedColormap
def make_zone_piechart(ecoregion_char, which_eoi:str, value:str, label:str, colors):
       # filter to an ecoregion
       ecoregion_char = ecoregion_char[ecoregion_char["ecoregion"]==which_eoi]

       # set the simplified veg map colors
       if isinstance(colors, dict):
              cmap = ListedColormap([colors[x] for x in ecoregion_char["Haz_Catego"].unique()])
       else:
              cmap=ListedColormap(colors)
              
       pie_colors = cmap(np.linspace(0., 1., len(ecoregion_char[value])))

       # Make pie chart
       fig, ax = plt.subplots(figsize=(10,10))
       ax.pie(ecoregion_char[value], colors = pie_colors,
              wedgeprops={'linewidth': 2.0, 'edgecolor': 'white'}, textprops={'size': 'x-large'}) #autopct='%1.1f%%', labels=ecoregion_char['lifeform+perc']
       ax.axis('equal') 
       ax.legend(labels = ecoregion_char[label], bbox_to_anchor=(0,1),frameon=False, labelcolor = "black", prop = {'weight':'bold', 'size': 20})
       return fig, ax

#### Sonoma Veg

In [52]:
# variables for iterating over
ecoregion_list = sonoma_veg_percs.ecoregion.unique()
veg_cols = ["peru", "blue", "red", "grey", "green", "bisque", "yellow"]

In [ ]:
make_zone_piechart(sonoma_veg_percs, "Bodega Coastal Hills", "acres", "lifeform+perc", veg_cols)

In [55]:
sonoma_veg_percs.ecoregion.unique()

array(['Bay Flats', 'Bodega Coastal Hills',
       'Coastal Franciscan Redwood Forest',
       'Fort Bragg/Fort Ross Terraces', 'Mayacmas Mountains',
       'Napa-Sonoma-Lake Volcanic Highlands',
       'Napa-Sonoma-Russian River Valleys',
       'North Coast Range Eastern Slopes',
       'Sonoma-Mendocino Mixed Forest'], dtype=object)

In [ ]:
# for loop to make and save a pie chart for every ecoregion
for eco in tqdm(ecoregion_list):
    fig, _ = make_zone_piechart(sonoma_veg_percs, eco, "percent_of_ecoregion", "lifeform+perc", veg_cols)
    snake_case_eco = eco.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
    fig.savefig(f"../output/graphs/{snake_case_eco}_veg_pie_blck.png", dpi=350)

#### Landuse

In [ ]:
# add landuse percentage columns and remove categories with less than 0.0 cover
landuse_percs["perc_round"] = landuse_percs['percent_of_ecoregion'].round(1)
landuse_percs = landuse_percs[landuse_percs["perc_round"] > 0.0]
landuse_percs["landuse+perc"] = landuse_percs['landuse'] + " (" + landuse["perc_round"].astype(str) + "%)"

In [ ]:
# for loop to make and save a pie chart for every ecoregion
for eco in tqdm(ecoregion_list):
    fig, _ = make_zone_piechart(landuse_percs, eco, "acres", "landuse+perc", landuse_colors)
    snake_case_eco = eco.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
    fig.savefig(f"../output/graphs/{snake_case_eco}_landuse_pie_blck.png", dpi=350)

#### Critical Facilities

In [31]:
#crit_facili["haz_category+number_facil"] = crit_facili["Haz_Catego"] + " (" + crit_facili["counts"].astype(str) + ")"
crit_facili = crit_facili.rename(columns = {"l4_simple": "ecoregion"})
crit_facili.head()

,Unnamed: 0,ecoregion,Haz_Catego,counts,haz_category+number_facil
0,0,Bay Flats,Communication,1,Communication (1)
1,1,Bay Flats,Energy,1,Energy (1)
2,2,Bay Flats,"Food, Water, Shelter",3,"Food, Water, Shelter (3)"
3,3,Bay Flats,Hazardous Material,99,Hazardous Material (99)
4,4,Bay Flats,Safety and Security,3,Safety and Security (3)


In [30]:
facility_colors = {
    'Communication': "#046C9A",
    'Energy': "#F98400",
    'Food, Water, Shelter': "#00A08A",
    'Hazardous Material': "#FF0000", 
    'Safety and Security': "#D69C4E", 
    'Transportation': "#ECCBAE",
    'Health and Medical':  "#ABDDDE"
}

In [ ]:
make_zone_piechart(crit_facili, "Bodega Coastal Hills", "counts", "haz_category+number_facil", facility_colors)

In [ ]:
ecoregion_list = crit_facili.ecoregion.unique()
for eco in tqdm(ecoregion_list):
    fig, _ = make_zone_piechart(crit_facili, eco, "counts", "Haz_Catego", facility_colors)
    snake_case_eco = eco.lower().replace(" ", "_").replace("/", "_").replace("-", "_")
    fig.savefig(f"../output/graphs/{snake_case_eco}_critical_facility_pie.png", dpi=350)

In [4]:
crit_facili.head()

,Unnamed: 0,l4_simple,Haz_Catego,counts
0,0,Bay Flats,Communication,1
1,1,Bay Flats,Energy,1
2,2,Bay Flats,"Food, Water, Shelter",3
3,3,Bay Flats,Hazardous Material,99
4,4,Bay Flats,Safety and Security,3


scratch:

In [ ]:
## Scratch ###
veg_top_5 = zones_veg_nat_top[["ecoregion", "natural_lifeform"]]
veg_top_5[["top"]] = "top_5"
veg_top_5.head()

# identify what vegetation types are the 5 most common
tester = zones_veg_nat.merge(veg_top_5, how='left', on = ['ecoregion', "natural_lifeform"])
tester["natural_lifeform_fix"] = np.select([tester['top']=='top_5'], [tester['natural_lifeform']], ['other'])
tester.head()